# eCommerce behavior data
<b>eCommerce behavior data from multi category store</b>
* <b>Data Source</b> : [kaggle](https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store?select=2019-Oct.csv), [REES46 Marketing Platform](https://rees46.com/)
* <b>About</b>: This dataset contains 285 million users' events from eCommerce website
* <b>File structure</b> :
|Property|Description|
|--------|-----------|
|event_time|Time when event happened at (in UTC)|
|event_type|view, cart, remove_from_cart, purchase|
|product_id|ID of a product|
|category_id|Product's category ID|
|category_code|Product's category taxonomy (code name) if it was possible to make it. Usually present for meaningful categories and skipped for different kinds of accessories.|
|brand|Downcased string of brand name. Can be missed.|
|price|Float price of a product. Present.|
|user_id|Permanent user ID.|
|user_session|Temporary user's session ID. Same for each user's session. Is changed every time user come back to online store from a long pause.|

## 라이브러리 로드

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os

## 데이터 전처리

### 데이터 일부만 가져오기
* 컬럼별 데이터 타입, 범위(min, max), nunique 확인  
* downcast 적용, 범주형 변경 가능한 컬럼 확인

In [3]:
file_path = glob('data/eCommerce_behavior_2019-Oct.csv')[0]
sneakpeak = pd.read_csv(file_path, nrows=5000)

In [83]:
print(sneakpeak.shape)
sneakpeak.head(3)

(5000, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8


In [84]:
sneakpeak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   event_time     5000 non-null   object 
 1   event_type     5000 non-null   object 
 2   product_id     5000 non-null   int64  
 3   category_id    5000 non-null   int64  
 4   category_code  3431 non-null   object 
 5   brand          4373 non-null   object 
 6   price          5000 non-null   float64
 7   user_id        5000 non-null   int64  
 8   user_session   5000 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 351.7+ KB


In [85]:
# 수치형 컬럼 min, max 확인 > price 컬럼 unsigned로 변환 가능
sneakpeak.describe()

,product_id,category_id,price,user_id
count,5.000000e+03,5.000000e+03,5000.000000,5.000000e+03
mean,9.959173e+06,2.055759e+18,310.726740,5.317238e+08
std,1.107719e+07,1.413163e+16,378.855998,1.747063e+07
min,1.002099e+06,2.053014e+18,0.000000,3.064418e+08
25%,1.005067e+06,2.053014e+18,65.120000,5.155540e+08
50%,4.804056e+06,2.053014e+18,166.145000,5.293785e+08
75%,1.580004e+07,2.053014e+18,385.827500,5.486073e+08
max,5.300000e+07,2.172371e+18,2574.070000,5.554633e+08


In [86]:
# object 컬럼 nunique값 확인 > category로 변환 가능할 컬럼 확인
sneakpeak.describe(include='O')

,event_time,event_type,category_code,brand,user_session
count,5000,5000,3431,4373,5000
unique,1540,3,93,465,1898
top,2019-10-01 02:27:05 UTC,view,electronics.smartphone,samsung,f13c75a5-7f6e-4467-a4e2-c1386c05b007
freq,16,4896,1568,680,24


In [87]:
# 수치형으로 되어있지만 범주형으로 바꿔도 될 것 같은 컬럼 nunique값 확인
print(f"""user_id : {sneakpeak['user_id'].nunique()},
product_id : {sneakpeak['product_id'].nunique()},
category_id : {sneakpeak['category_id'].nunique()}""")

user_id : 1747,
product_id : 2305,
category_id : 304


### downcast 적용 (데이터 일부에만 적용해보기)

1. 범주형으로 변경
* event_type : object > categorial
* category_code : object > categorial
* category_id : int64 > categorial
* brand : object> categorial  
* user_id : int64 > categorial

2. 수치형 downcast
* product_id : int64 > unsigned int32
* price : float64 > unsigned

3. 변경 안함
* user_session : object

In [13]:
def downcast_dataframe(df) :
    """
    return : df를 받으면 컬럼별로 범주형으로 변환 혹은 downcast된 df를 리턴
    pirnt : 얼만큼의 memory_usage가 감소했는지 출력
    """
    # 기존 메모리 사용량 변수로 저장
    memory_usage1 = df.memory_usage().sum()

    for col in df.columns : 
        # downcast 적용
        df['event_time'] = pd.to_datetime(df['event_time']) # datetime64[ns]으로 변경
        df['price'] = pd.to_numeric(df['price'], downcast='float') # float32로 변경
        df['product_id'] = pd.to_numeric(df['product_id'], downcast='unsigned') # uint32로 변경

        # category 변경
        df['user_id'] = df['user_id'].astype('category')
        df['category_id'] = df['category_id'].astype('category')
        df['category_code'] = df['category_code'].astype('category')
        df['brand'] = df['brand'].astype('category')
        df['user_session'] = df['user_session'].astype('category')
    
    # 줄어든 메모리 사용량 변수로 저장 > 절약된 수치 출력
    memory_usage2 = df.memory_usage().sum()
    print(f"{round((memory_usage1-memory_usage2) / memory_usage1, 2)*100}% memory saved")

    return df

In [118]:
sneakpeak = downcast_dataframe(sneakpeak)

19.0% memory saved


In [119]:
sneakpeak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   event_time     5000 non-null   datetime64[ns, UTC]
 1   event_type     5000 non-null   object             
 2   product_id     5000 non-null   uint32             
 3   category_id    5000 non-null   category           
 4   category_code  3431 non-null   category           
 5   brand          4373 non-null   category           
 6   price          5000 non-null   float32            
 7   user_id        5000 non-null   uint32             
 8   user_session   5000 non-null   category           
dtypes: category(4), datetime64[ns, UTC](1), float32(1), object(1), uint32(2)
memory usage: 283.4+ KB


### 전체 csv 파일 로드

In [4]:
csv_path_oct = 'data/eCommerce_behavior_2019-Oct.csv'
csv_path_nov = 'data/eCommerce_behavior_2019-Nov.csv'

In [121]:
df_oct = pd.read_csv(glob(csv_path_oct)[0])
df_nov = pd.read_csv(glob(csv_path_nov)[0])

In [122]:
df = pd.concat([df_oct, df_nov])

In [123]:
print(df.info())
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109950743 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 8.2+ GB
None


(109950743, 9)

### downcast 적용 (전체 csv파일에 적용)

In [125]:
df_1 = downcast_dataframe(df_oct)
df_2 = downcast_dataframe(df_nov)

37.0% memory saved
34.0% memory saved


In [126]:
df = pd.concat([df_1, df_2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109950743 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     uint32             
 3   category_id    int64              
 4   category_code  object             
 5   brand          object             
 6   price          float32            
 7   user_id        uint32             
 8   user_session   object             
dtypes: datetime64[ns, UTC](1), float32(1), int64(1), object(4), uint32(2)
memory usage: 7.0+ GB


### parquet 으로 저장

In [ ]:
parquet_path = csv_path_oct[:-7] + 'parquet'
df.to_parquet(parquet_path, compression='gzip')

In [6]:
df = pd.read_parquet(parquet_path)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109950743 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     int64              
 3   category_id    int64              
 4   category_code  object             
 5   brand          object             
 6   price          float32            
 7   user_id        int64              
 8   user_session   object             
dtypes: datetime64[ns, UTC](1), float32(1), int64(3), object(4)
memory usage: 7.8+ GB


In [14]:
df = downcast_dataframe(df)
df.info()

-3.0% memory saved
<class 'pandas.core.frame.DataFrame'>
Int64Index: 109950743 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     uint32             
 3   category_id    category           
 4   category_code  category           
 5   brand          category           
 6   price          float32            
 7   user_id        category           
 8   user_session   category           
dtypes: category(5), datetime64[ns, UTC](1), float32(1), object(1), uint32(1)
memory usage: 5.6+ GB
